In [1]:
from scipy.spatial import Delaunay
import py4cytoscape as p4c
import numpy as np
import networkx as nx
import optuna
from egraph import Graph, Coordinates, Rng, SparseSgd
import matplotlib.pyplot as plt
import json
import math

%matplotlib inline


/Users/fuga_takata/dev/vdslab-project/hyperparameter_optimization/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
EDGE_WEIGHT = 30
K_FROM = 1
K_TO = 20

LARGE_DATASET_NAMES = [
    '3elt.json',
    '1138_bus.json',
    'dwt_1005.json',
    'dwt_2680.json',
    'poli.json',
    'qh882.json',
    'USpowerGrid.json',
]
SMALL_DATASET_NAMES = [
    'bull.json',
    'chvatal.json',
    'cubical.json',
    'davis_southern_women.json',
    'desargues.json',
    'diamond.json',
    'dodecahedral.json',
    'florentine_families.json',
    'frucht.json',
    'heawood.json',
    'hoffman_singleton.json',
    'house_x.json',
    'house.json',
    'icosahedral.json',
    'karate_club.json',
    'krackhardt_kite.json',
    'les_miserables.json',
    'moebius_kantor.json',
    'octahedral.json',
    'pappus.json',
    'petersen.json',
    'sedgewick_maze.json',
    'tutte.json',
]


In [3]:
# グラフの生成・読み込み

def generate_graph_from_nx_graph(nx_graph):
    graph = Graph()

    indices = {}
    for u in nx_graph.nodes:
        indices[u] = graph.add_node(u)
    for u, v in nx_graph.edges:
        graph.add_edge(indices[u], indices[v], (u, v))

    return graph, indices


def graph_preprocessing(nx_graph):
    nx_graph = nx.Graph(nx_graph)

    # グラフを無向グラフに
    nx_graph = nx_graph.to_undirected()

    # エッジの自己ループを除去
    nx_graph.remove_edges_from(list(nx.selfloop_edges(nx_graph)))

    # 最大連結成分を用いる
    largest_cc = max(nx.connected_components(nx_graph), key=len)
    largest_cc_graph = nx_graph.subgraph(largest_cc)

    new_graph = nx.Graph()
    nodes = [str(node_id) for node_id in largest_cc_graph.nodes]
    new_graph.add_nodes_from(nodes)

    # エッジにidと重みを追加
    for i, edge in enumerate(largest_cc_graph.edges):
        new_graph.add_edge(str(edge[0]), str(
            edge[1]), weight=EDGE_WEIGHT, id=str(i))
        # weighted_edges.append((str(edge[0]), str(edge[1]), EDGE_WEIGHT))

    return new_graph


In [4]:
# shape_based_metrics
def generate_delaunay_triangulation_graph(pos):
    index_id_map = {}
    pos_array = []
    for index, node_id in enumerate(pos):
        positions = pos[node_id]
        position = [positions[0], positions[1]]
        pos_array.append(position)
        index_id_map[index] = node_id

    pos_ndarray = np.array(pos_array)
    delaunay = Delaunay(pos_ndarray)

    delaunay_triangulation_graph = nx.Graph()

    nodes = [node_id for node_id in pos]
    delaunay_triangulation_graph.add_nodes_from(nodes)

    weighted_edges = []
    for n in delaunay.simplices:
        n0 = n[0]
        n1 = n[1]
        n2 = n[2]
        weighted_edges.append(
            (index_id_map[n0], index_id_map[n1], EDGE_WEIGHT))
        weighted_edges.append(
            (index_id_map[n0], index_id_map[n2], EDGE_WEIGHT))
        weighted_edges.append(
            (index_id_map[n1], index_id_map[n2], EDGE_WEIGHT))
    delaunay_triangulation_graph.add_weighted_edges_from(weighted_edges)

    return delaunay_triangulation_graph


def jaccard_similarity_sum(nx_graph, nx_shape_graph):
    j_s_sum = 0
    for node in nx_graph.nodes:
        g_n = [n for n in nx_graph.neighbors(node)]
        s_n = [n for n in nx_shape_graph.neighbors(node)]
        and_n = list(set(g_n) & set(s_n))
        or_n = list(set(g_n + s_n))

        j_s_sum += len(and_n) / len(or_n)

    return j_s_sum / len(nx_graph.nodes)

# maximize


def shape_based_metrics(nx_graph, pos):
    nx_shape_graph = generate_delaunay_triangulation_graph(pos)

    return jaccard_similarity_sum(nx_graph, nx_shape_graph)


In [5]:
# minimize
def stress(nx_graph, pos, K=1, L=1):
    shortest_paths = dict(nx.all_pairs_dijkstra_path_length(nx_graph))
    s = 0
    node_ids = sorted([node_id for node_id in pos])
    for i, sid in enumerate(node_ids):
        for tid in node_ids[i + 1:]:
            norm = np.linalg.norm(np.array(pos[sid]) - np.array(pos[tid]))
            dij = shortest_paths[sid][tid]
            lij = L * dij
            kij = K * dij
            e = (kij * ((norm - lij) ** 2)) / 2

            s += e
    return s


In [6]:
# minimize
def ideal_edge_length(nx_graph, pos):
    s = 0
    for source, target, data in nx_graph.edges(data=True):
        weight = data['weight'] if 'weight' in data else 1
        shortest_path_length = nx.shortest_path_length(
            nx_graph, source, target, data['weight'] if 'weight' in data else None)

        lij = shortest_path_length * weight
        dist = np.linalg.norm(np.array(pos[source]) - np.array(pos[target]))
        s += ((dist - lij) / lij) ** 2

    return s


In [7]:
def is_edge_crossing(p1, p2, p3, p4):
    tc1 = (p1[0] - p2[0]) * (p3[1] - p1[1]) + (p1[1] - p2[1]) * (p1[0] - p3[0])
    tc2 = (p1[0] - p2[0]) * (p4[1] - p1[1]) + (p1[1] - p2[1]) * (p1[0] - p4[0])
    td1 = (p3[0] - p4[0]) * (p1[1] - p3[1]) + (p3[1] - p4[1]) * (p3[0] - p1[0])
    td2 = (p3[0] - p4[0]) * (p2[1] - p3[1]) + (p3[1] - p4[1]) * (p3[0] - p2[0])
    return tc1 * tc2 < 0 and td1 * td2 < 0


def edge_crossing_finder(nx_graph, pos):
    edges = {}
    for s1, t1, attr1 in nx_graph.edges(data=True):
        id1 = attr1['id']
        if id1 not in edges:
            edges[id1] = {}
        for s2, t2, attr2 in nx_graph.edges(data=True):
            id2 = attr2['id']
            crossing = is_edge_crossing(pos[s1], pos[t1], pos[s2], pos[t2])
            edges[id1][id2] = crossing

    return edges


In [8]:
# minimize
def crossing_number(nx_graph, pos, edge_crossing=None):
    if edge_crossing is None:
        edge_crossing = edge_crossing_finder(nx_graph, pos)

    s = 0
    node_ids = sorted([node_id for node_id in edge_crossing])
    for i, sid in enumerate(node_ids):
        for tid in node_ids[i + 1:]:
            if edge_crossing[sid][tid]:
                s += 1

    return s


In [9]:
# maximize
def crossing_angle_maximization(nx_graph, pos, edge_crossing=None):
    if edge_crossing is None:
        edge_crossing = edge_crossing_finder(nx_graph, pos)
    s = 0
    for s1, t1, attr1 in nx_graph.edges(data=True):
        id1 = attr1['id']
        for s2, t2, attr2 in nx_graph.edges(data=True):
            id2 = attr2['id']
            if edge_crossing[id1][id2] or edge_crossing[id2][id1]:
                e1 = np.array(pos[s1]) - np.array(pos[t1])
                e2 = np.array(pos[s2]) - np.array(pos[t2])
                s += np.dot(e1, e2) / (np.linalg.norm(e1) * np.linalg.norm(e2))

    return s


In [10]:
def gravity_center(pos):
    gx = 0
    gy = 0
    for node_id in pos:
        x, y = pos[node_id]
        gx += x
        gy += y

    gx /= len(pos)
    gy /= len(pos)

    return gx, gy

# maximize


def aspect_ratio(pos):
    gravity_centered_pos = []
    gx, gy = gravity_center(pos)
    for node_id in pos:
        x, y = pos[node_id]
        gravity_centered_pos.append([x - gx, y - gy])
    gravity_centered_pos = np.array(gravity_centered_pos)
    _, s, _ = np.linalg.svd(gravity_centered_pos, full_matrices=True)
    a = max(s)
    b = min(s)

    return b / a


In [11]:
def get_max_degree(nx_graph):
    md = 0
    for node_id in nx_graph.nodes:
        d = nx_graph.degree[node_id]
        if md < d:
            md = d

    return md

# maximize
# すべてのノードについて、あるノードに入射するエッジ同士のなす角度が最も小さいもの


def angular_resolution(nx_graph, pos):
    edges = {}
    for s, t in nx_graph.edges:
        if s not in edges:
            edges[s] = {}
        if t not in edges:
            edges[t] = {}
        edges[s][t] = True
        edges[t][s] = True

    ls = 0
    for s in pos:
        if s not in edges:
            continue
        l = 2 * math.pi
        ts = sorted([node_id for node_id in edges[s]])
        for i, t1 in enumerate(ts):
            e1 = np.array(pos[s]) - np.array(pos[t1])
            for t2 in ts[i + 1:]:
                if t1 == t2 or s == t1 or s == t2:
                    continue
                e2 = np.array(pos[s]) - np.array(pos[t2])
                angle = math.acos(
                    np.dot(e1, e2) / (np.linalg.norm(e1) * np.linalg.norm(e2)))
                if angle < l:
                    l = angle
        ls += l
    return ls


In [12]:
# maximize
# ノードのユークリッド距離として定義される。stressなどと一致させるために正規化するらしいけど多分必要ない
def node_resolution(pos):
    nodes = sorted([node_id for node_id in pos])
    r = float('inf')
    ds = []
    for i, sid in enumerate(nodes):
        for tid in nodes[i + 1:]:
            dist = np.linalg.norm(np.array(pos[sid]) - np.array(pos[tid]))
            ds.append(dist)
            if dist < r:
                r = dist
    return r


In [13]:
# minimize
# エッジを直径とした円の内部にノードを含まないようにしたい。
# そこで内部に含まれる点
def gabriel_graph_property(nx_graph, pos):
    s = 0
    for e in nx_graph.edges:
        e1, e2 = e
        x1 = np.array(pos[e1])
        x2 = np.array(pos[e2])
        rij = np.linalg.norm(x1 - x2) / 2
        cij = (np.array(x1) + np.array(x2)) / 2
        for node_id in nx_graph.nodes:
            if e1 == node_id or e2 == node_id:
                continue
            s += max(0, rij - np.linalg.norm(np.array(pos[node_id] - cij)))

    return s


In [14]:
# グラフの描画

def draw_graph(graph, indices, params):
    drawing = Coordinates.initial_placement(graph)
    rng = Rng.seed_from(0)  # random seed
    sgd = SparseSgd(
        graph,
        lambda _: params['edge_length'],  # edge length
        params['number_of_pivots'],  # number of pivots
        rng,
    )
    scheduler = sgd.scheduler(
        params['number_of_iterations'],  # number of iterations
        params['eps'],  # eps: eta_min = eps * min d[i, j] ^ 2
    )

    def step(eta):
        sgd.shuffle(rng)
        sgd.apply(drawing, eta)

    scheduler.run(step)

    pos = {u: (drawing.x(i), drawing.y(i)) for u, i in indices.items()}

    return pos


In [15]:
def calc_quality_metrics(nx_graph, pos):
    edge_crossing = edge_crossing_finder(nx_graph, pos)

    quality_metrics = {}
    quality_metrics['stress'] = stress(nx_graph, pos)
    quality_metrics['ideal_edge_length'] = ideal_edge_length(nx_graph, pos)
    quality_metrics['shape_based_metrics'] = shape_based_metrics(nx_graph, pos)
    quality_metrics['crossing_number'] = crossing_number(
        nx_graph, pos, edge_crossing)
    quality_metrics['crossing_angle_maximization'] = crossing_angle_maximization(
        nx_graph, pos, edge_crossing)
    quality_metrics['aspect_ratio'] = aspect_ratio(pos)
    quality_metrics['angular_resolution'] = angular_resolution(nx_graph, pos)
    quality_metrics['node_resolution'] = node_resolution(pos)
    quality_metrics['gabriel_graph_property'] = gabriel_graph_property(
        nx_graph, pos)

    return quality_metrics


In [16]:
DATASET_NAMES = None
DATASET_NAME = 'USpowerGrid.json'

In [17]:
global_pos = []


def objective_wrapper(nx_graph, graph, indices, quality_metrics_names):
    def objective(trial: optuna.Trial):
        params = {
            'edge_length': trial.suggest_int('edge_length', 1, 100),
            'number_of_pivots': trial.suggest_int('number_of_pivots', 1, len(nx_graph.nodes)),
            'number_of_iterations': trial.suggest_int('number_of_iterations', 1, 1000),
            'eps': trial.suggest_float('eps', 0.01, 1)
        }

        pos = draw_graph(graph, indices, params)
        trial.set_user_attr('pos', pos)
        global_pos.append(pos)

        quality_metrics = calc_quality_metrics(nx_graph, pos)
        return tuple([quality_metrics[name] for name in quality_metrics_names])
    return objective


In [18]:
# LAYOUT_NAME = 'kamada-kawai'


# def objective_wrapper(nx_graph, graph, indices, quality_metrics_names):
#     def objective(trial: optuna.Trial):
#         p4c.delete_all_networks()
#         p4c.create_network_from_networkx(nx_graph)

#         params = {
#             'm_averageIterationsPerNode': trial.suggest_float('m_averageIterationsPerNode', 0, 100),
#             'm_nodeDistanceStrengthConstant': trial.suggest_float('m_nodeDistanceStrengthConstant', 0, 50),
#             'm_nodeDistanceRestLengthConstant': trial.suggest_float('m_nodeDistanceRestLengthConstant', 0, 100),
#             'm_disconnectedNodeDistanceSpringStrength': trial.suggest_float('m_disconnectedNodeDistanceSpringStrength', 0, 1),
#             'm_disconnectedNodeDistanceSpringRestLength': trial.suggest_float('m_disconnectedNodeDistanceSpringRestLength', 0, 10000),
#             'm_anticollisionSpringStrength': trial.suggest_float('m_anticollisionSpringStrength', 0, 1),
#             'm_layoutPass': trial.suggest_int('m_layoutPass', 0, 5),
#             'singlePartition': False,
#             'unweighted': False,
#             'randomize': False
#         }

#         p4c.set_layout_properties(
#             layout_name=LAYOUT_NAME, properties_dict=params)
#         p4c.layout_network(layout_name=LAYOUT_NAME)
#         pos = p4c.get_node_position()

#         pos = draw_graph(graph, indices, params)

#         quality_metrics = calc_quality_metrics(nx_graph, pos)
#         return tuple([quality_metrics[name] for name in quality_metrics_names])
#     return objective


In [19]:
dir = 'lib/egraph-rs/js/dataset/'
with open(dir + DATASET_NAME) as f:
    graph_data = json.load(f)
nx_graph = graph_preprocessing(nx.node_link_graph(graph_data))
graph, indices = generate_graph_from_nx_graph(nx_graph)


In [20]:
DIRECTION = {'max': 'maximize', 'min': "minimize"}

quality_metrics_direction = {}
quality_metrics_direction['stress'] = DIRECTION['min']
quality_metrics_direction['ideal_edge_length'] = DIRECTION['min']
quality_metrics_direction['shape_based_metrics'] = DIRECTION['max']
quality_metrics_direction['crossing_number'] = DIRECTION['min']
quality_metrics_direction['crossing_angle_maximization'] = DIRECTION['max']
quality_metrics_direction['aspect_ratio'] = DIRECTION['max']
quality_metrics_direction['angular_resolution'] = DIRECTION['max']
quality_metrics_direction['node_resolution'] = DIRECTION['max']
quality_metrics_direction['gabriel_graph_property'] = DIRECTION['min']

quality_metrics_names = sorted([k for k in quality_metrics_direction])


In [21]:
# N_TRIALS = 20
# study = optuna.create_study(
#     directions=[quality_metrics_direction[name] for name in quality_metrics_names])

# study.optimize(objective_wrapper(nx_graph, graph, indices,
#                quality_metrics_names), n_trials=N_TRIALS, show_progress_bar=True)


In [22]:
# data = []

# for best_trial in study.best_trials:
#     params = best_trial.params

#     data.append(params)

# filename = f'{DATASET_NAME.split(".")[0]}_params.json'
# with open(f'data/{filename}', mode='w') as f:
#     json.dump(data, f)


In [23]:
# data = {}
# data['results'] = []
# data['quality_metrics_names'] = quality_metrics_names
# for best_trial in study.best_trials:
#     values = best_trial.values
#     data['results'].append(values)

# filename = f'{DATASET_NAME.split(".")[0]}_results.json'
# with open(f'data/{filename}', mode='w') as f:
#     json.dump(data, f)


In [24]:
# import random

# data = {}
# data['quality_metrics_names'] = quality_metrics_names
# data['result'] = []
# data['params'] = []
# data['pos'] = []

# trials = 5

# for _ in range(trials):
#     params = {
#         'edge_length': random.randint(1, 100),
#         'number_of_pivots': random.randint(1, len(nx_graph.nodes)),
#         'number_of_iterations': random.randint(1, 1000),
#         'eps': random.uniform(0.01, 1)
#     }

#     pos = draw_graph(graph, indices, params)

#     quality_metrics = calc_quality_metrics(nx_graph, pos)
#     values = [quality_metrics[name] for name in quality_metrics_names]

#     data['result'].append(values)
#     data['params'].append(params)
#     data['pos'].append(pos)

# filename = f'{DATASET_NAME.split(".")[0]}_randomized_data.json'
# with open(f'data/{filename}', mode='w') as f:
#     json.dump(data, f)


In [25]:
with open(f'data/{DATASET_NAME.split(".")[0]}_optimized_data_nopos.json') as f:
    jsondata = json.load(f)

dir = 'lib/egraph-rs/js/dataset/'
with open(dir + DATASET_NAME) as f:
    graph_data = json.load(f)
nx_graph = graph_preprocessing(nx.node_link_graph(graph_data))
graph, indices = generate_graph_from_nx_graph(nx_graph)


jsondata['pos'] = []
for p in jsondata['params']:
    pos = draw_graph(graph, indices, p)
    jsondata['pos'].append(pos)
    print(p)


filename = f'{DATASET_NAME.split(".")[0]}_optimized_data.json'
with open(f'data/{filename}', mode='w') as f:
    json.dump(jsondata, f)


{'edge_length': 60, 'number_of_pivots': 1322, 'number_of_iterations': 681, 'eps': 0.8502822373438783}
{'edge_length': 26, 'number_of_pivots': 281, 'number_of_iterations': 867, 'eps': 0.4001671578513263}
{'edge_length': 69, 'number_of_pivots': 1898, 'number_of_iterations': 517, 'eps': 0.33002499120244505}
{'edge_length': 30, 'number_of_pivots': 1502, 'number_of_iterations': 482, 'eps': 0.4768008887987786}
{'edge_length': 69, 'number_of_pivots': 2915, 'number_of_iterations': 181, 'eps': 0.03620086344394183}
{'edge_length': 92, 'number_of_pivots': 2699, 'number_of_iterations': 266, 'eps': 0.023059806189874736}
{'edge_length': 85, 'number_of_pivots': 2523, 'number_of_iterations': 908, 'eps': 0.9593103420065461}
{'edge_length': 58, 'number_of_pivots': 4016, 'number_of_iterations': 568, 'eps': 0.046034308378419764}
{'edge_length': 47, 'number_of_pivots': 1980, 'number_of_iterations': 451, 'eps': 0.21243852974511238}
{'edge_length': 43, 'number_of_pivots': 260, 'number_of_iterations': 468, 'e

In [26]:
# data = []

# for best_trial in study.best_trials:
#     params = best_trial.params

#     data.append(params)

# filename = f'{DATASET_NAME.split(".")[0]}_random_params.json'
# with open(f'data/{filename}', mode='w') as f:
#     json.dump(data, f)


In [27]:
# data = {}
# data['results'] = []
# data['quality_metrics_names'] = quality_metrics_names
# for best_trial in study.best_trials:
#     values = best_trial.values
#     data['results'].append(values)

# filename = f'{DATASET_NAME.split(".")[0]}_random_results.json'
# with open(f'data/{filename}', mode='w') as f:
#     json.dump(data, f)


In [28]:
# data = {}
# data['results'] = []
# data['quality_metrics_names'] = quality_metrics_names
# for best_trial in study.best_trials:
#     values = best_trial.values
#     data['results'].append(values)

# filename = f'{DATASET_NAME.split(".")[0]}_random_pos.json'
# with open(f'data/{filename}', mode='w') as f:
#     json.dump(data, f)
